<a href="https://colab.research.google.com/github/sergekamanzi/Fraud-Detection/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Nadam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from focal_loss import BinaryFocalLoss
import seaborn as sns
import matplotlib.pyplot as plt

In [62]:
data = pd.read_csv('/content/fraud detection.csv')
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [63]:
data.shape

(6362620, 11)

In [ ]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the 'BMIcase' and 'Gender' columns
data['type'] = label_encoder.fit_transform(data['type'])
data['nameOrig'] = label_encoder.fit_transform(data['nameOrig'])
data['nameDest'] = label_encoder.fit_transform(data['nameDest'])

In [ ]:
# Feature Engineering: Create meaningful features
data['balance_change_orig'] = data['oldbalanceOrg'] - data['newbalanceOrig']
data['balance_change_dest'] = data['oldbalanceDest'] - data['newbalanceDest']
data['transaction_ratio_orig'] = data['amount'] / (data['oldbalanceOrg'] + 1)  # Avoid division by zero
data['transaction_ratio_dest'] = data['amount'] / (data['oldbalanceDest'] + 1)

In [ ]:
%%capture
data.head()

In [ ]:
# Handle missing values (impute with median)
data.fillna(data.median(), inplace=True)

In [ ]:
%%capture
# Checking for NAN values
data.isna().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = data.corr()
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Heatmap of Dataset Correlations')
plt.show()

In [ ]:
# Drop 'nameDest' as it doesn't provide useful information
data.drop(columns=['nameDest', 'isFlaggedFraud'], inplace=True)

In [ ]:
# Split features and target
X = data.drop(columns=['isFraud'])
Y = data['isFraud']

In [ ]:
# to see how many 1s and 0s in isFraud
fraud_count = data['isFraud'].value_counts()[1]
print(f"{fraud_count} fraudulent transactions in the dataset.")
fraud_count1 = data['isFraud'].value_counts()[0]
print(f"{fraud_count1} non fraudulent transactions in the dataset.")

In [ ]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split into train, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X_scaled, Y, test_size=0.3, random_state=42, stratify=Y)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42, stratify=Y_temp)

# Apply SMOTE to handle class imbalance
smote = SMOTE(sampling_strategy=0.5, random_state=42)  # Adjust strategy as needed
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

# Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(Y_train_resampled), y=Y_train_resampled)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:
# Define the Model with Nadam, L2 Regularization & Batch Normalization
model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train_resampled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile model with Nadam and Focal Loss
optimizer = Nadam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=BinaryFocalLoss(gamma=2), metrics=['accuracy'])

In [ ]:
# Train the Model
history = model.fit(X_train_resampled, Y_train_resampled,
                    validation_data=(X_val, Y_val),
                    epochs=50, batch_size=256,
                    class_weight=class_weights_dict,
                    verbose=1)

In [ ]:
# Evaluate the model on test set
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
# Calculate Accuracy and F1 Score
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Classification Report
print("\nClassification Report:\n", classification_report(Y_test, Y_pred))

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()